In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
################################
# Imports
################################

import pandas as pd
import h5py
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import glob
import re
from multiprocessing import cpu_count
from multiprocessing import Pool

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


import os
import sys
import subprocess
import random
from random import sample
from tqdm import tqdm
from functools import partial

from sklearn.metrics import confusion_matrix


import librosa

In [3]:
################################
# configs
################################

#################
# spectrogram
#################

# Mean of flute = -6.438913345336914
# Median of flute = -6.118330955505371
# stdDev of flute = 4.377377986907959
# Max of flute = 1.8442230224609375
# Min of flute = -39.0754280090332

# Mean of piano = -6.015857219696045
# Median of piano = -5.299488544464111
# stdDev of piano = 4.420877456665039
# Max of piano = 1.5825170278549194
# Min of piano = -40.520179748535156

spectrogramStats = {
                    'flute': {'mean': -6.438913345336914, 'median': -6.118330955505371, 'stdDev': 4.377377986907959, 'max': 1.8442230224609375, 'min': -39.0754280090332},
                    'piano': {'mean': -6.015857219696045, 'median': -5.299488544464111, 'stdDev': 4.420877456665039, 'max': 1.5825170278549194, 'min': -40.520179748535156}
                    }

# standardizationStyleOptions = normal, logNormal, uniform
# following needs to be 'SUBTRACTED' from the data
centerOffset = {
                'flute': {'normal': spectrogramStats['flute']['mean'], 'logNormal': spectrogramStats['flute']['mean'], 'uniform': (spectrogramStats['flute']['min'] + spectrogramStats['flute']['max'])/2},
                'piano': {'normal': spectrogramStats['piano']['mean'], 'logNormal': spectrogramStats['piano']['mean'], 'uniform': (spectrogramStats['piano']['min'] + spectrogramStats['piano']['max'])/2}
                }

# following needs to be 'DIVIDED' to the data
divFactor = {
            'flute': {'normal': 3*spectrogramStats['flute']['stdDev'], 'logNormal': (1.1*spectrogramStats['flute']['max'] - spectrogramStats['flute']['mean']) , 'uniform': 1*(spectrogramStats['flute']['max'] - spectrogramStats['flute']['min'])/2},
            'piano': {'normal': 3*spectrogramStats['piano']['stdDev'], 'logNormal': (1.1*spectrogramStats['piano']['max'] - spectrogramStats['piano']['mean']) , 'uniform': 1*(spectrogramStats['piano']['max'] - spectrogramStats['piano']['min'])/2}
            }


####################
# training params
####################
STANDARDIZATION_STYLE = 'uniform'
BATCH_SIZE = 4
NUM_WORKERS = 2
NUM_EPOCHS = 10
LEARNING_RATE = 2e-4
LAMBDA_CYCLE = 10
LAMBDA_IDENTITY = 1
GRADIENT_PENALTY = 0
ADAM_BETA1 = 0.5
ADAM_BETA2 = 0.999
NUM_RESIDUALS = 6

####################
# checkpoint options
####################
SAVE_CHECKPOINTS = True

####################
# find GPU device
####################
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (f'Device = {DEVICE}, # of CUDA devices = {torch.cuda.device_count()}')


###################
# add fileTag
###################
fileTag = f'lr_{LEARNING_RATE}_cyc_{LAMBDA_CYCLE}_id_{LAMBDA_IDENTITY}_b1_{ADAM_BETA1}_b2_{ADAM_BETA2}_numRes_{NUM_RESIDUALS}_gp_{GRADIENT_PENALTY}_stdStyle_{STANDARDIZATION_STYLE}'


Device = cuda:0, # of CUDA devices = 1


In [4]:
################################
# discriminator
################################

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, stride, 1, bias=True, padding_mode="reflect"),
            nn.InstanceNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )

    def forward(self, x):
        return self.conv(x)


class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(
                in_channels,
                features[0],
                kernel_size=4,
                stride=2,
                padding=1,
                padding_mode="reflect"
            ),
            nn.LeakyReLU(0.2),
        )

        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(Block(in_channels, feature, stride=1 if feature==features[-1] else 2))
            in_channels = feature
        layers.append(nn.Conv2d(in_channels, 1, kernel_size=4, stride=1, padding=1, padding_mode="reflect"))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        x = self.initial(x)
        return torch.sigmoid(self.model(x))
        
    
def test():
    x = torch.randn((5,1,336,256))
    model = Discriminator(in_channels=1)
    print (model)
    preds = model(x)
    print(preds.shape)

test()

Discriminator(
  (initial): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (model): Sequential(
    (0): Block(
      (conv): Sequential(
        (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): LeakyReLU(negative_slope=0.2)
      )
    )
    (1): Block(
      (conv): Sequential(
        (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): LeakyReLU(negative_slope=0.2)
      )
    )
    (2): Block(
      (conv): Sequential(
        (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(512, eps=1e-05, momentum

In [5]:
################################
# generator
################################

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, use_act=True, **kwargs):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, padding_mode="reflect", **kwargs)
            if down
            else nn.ConvTranspose2d(in_channels, out_channels, **kwargs),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True) if use_act else nn.Identity()
        )
    
    def forward(self, x):
        return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            ConvBlock(channels, channels, kernel_size=3, padding=1),
            ConvBlock(channels, channels, use_act=False, kernel_size=3, padding=1),
        )
    
    def forward(self, x):
        return x + self.block(x)


class Generator(nn.Module):
    def __init__(self, img_channels, num_features = 64, num_residuals=6):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(img_channels, num_features, kernel_size=7, stride=1, padding=3, padding_mode="reflect"),
            nn.ReLU(inplace=True),
        )
        self.down_blocks = nn.ModuleList(
            [
                ConvBlock(num_features, num_features*2, kernel_size=3, stride=2, padding=1),
                ConvBlock(num_features*2, num_features*4, kernel_size=3, stride=2, padding=1),
            ]
        )

        self.residual_blocks = nn.Sequential(
            *[ResidualBlock(num_features*4) for _ in range(num_residuals)]
        )

        self.up_blocks = nn.ModuleList(
            [
                ConvBlock(num_features*4, num_features*2, down=False, kernel_size=3, stride=2, padding=1, output_padding=1),
                ConvBlock(num_features*2, num_features, down=False, kernel_size=3, stride=2, padding=1, output_padding=1),
            ]
        )

        self.last = nn.Conv2d(num_features, img_channels, kernel_size=7, stride=1, padding =3, padding_mode="reflect")
    
    def forward(self, x):
        x = self.initial(x)
        for layer in self.down_blocks:
            x = layer(x)
        x = self.residual_blocks(x)
        for layer in self.up_blocks:
            x = layer(x)
        return torch.tanh(self.last(x))
    
def test():
    x = torch.randn((5,1,336,256))
    model = Generator(img_channels=1, num_features = 16, num_residuals=9)
    print (model)
    gen = model(x)    
    print(gen.shape)

    
test()

Generator(
  (initial): Sequential(
    (0): Conv2d(1, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), padding_mode=reflect)
    (1): ReLU(inplace=True)
  )
  (down_blocks): ModuleList(
    (0): ConvBlock(
      (conv): Sequential(
        (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): ReLU(inplace=True)
      )
    )
    (1): ConvBlock(
      (conv): Sequential(
        (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): ReLU(inplace=True)
      )
    )
  )
  (residual_blocks): Sequential(
    (0): ResidualBlock(
      (block): Sequential(
        (0): ConvBlock(
          (conv): Sequential(
            (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), 

In [6]:
# data loader

class Dataset(torch.utils.data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, rootDir):
        'Initialization'
        self.rootDir = rootDir
        self.fileList = os.listdir(rootDir)        
        

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.fileList)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        X = np.load(os.path.join(self.rootDir, self.fileList[index]))
        
        return X

In [7]:
##########################
# dataset class
##########################

class PianoFluteDataset(Dataset):
    def __init__(self, root_piano, root_flute, standardizationStyle):
        self.root_piano = root_piano
        self.root_flute = root_flute

        random.seed(2)
        self.piano_images = os.listdir(root_piano)
        self.flute_images = sample(os.listdir(root_flute), len(os.listdir(root_flute)))
        
        self.length_dataset = max(len(self.piano_images), len(self.flute_images))

        self.piano_dataset_length = len(self.piano_images)
        self.flute_dataset_length = len(self.flute_images)
        
        self.standardizationStyle = standardizationStyle
        
        # default to normal for standardizing the data
        self.offsetFlute = centerOffset['flute']['normal']
        self.offsetPiano = centerOffset['piano']['normal']
        
        self.divFlute = divFactor['flute']['normal']
        self.divPiano = divFactor['piano']['normal']
        
        if standardizationStyle == 'uniform':
            print ('Using uniform assumption...')
            self.offsetFlute = centerOffset['flute']['uniform']
            self.offsetPiano = centerOffset['piano']['uniform']
        
            self.divFlute = divFactor['flute']['uniform']
            self.divPiano = divFactor['piano']['uniform']
            
        else:
            print ('Using logNormal assumption...')
            self.offsetFlute = centerOffset['flute']['logNormal']
            self.offsetPiano = centerOffset['piano']['logNormal']
        
            self.divFlute = divFactor['flute']['logNormal']
            self.divPiano = divFactor['piano']['logNormal']
        

    def __len__(self):
        return self.length_dataset

    def __getitem__(self, index):
        
        flute_image = self.flute_images[index % self.flute_dataset_length]
        piano_image = self.piano_images[index % self.piano_dataset_length]

        flute_path = os.path.join(self.root_flute, flute_image)
        piano_path = os.path.join(self.root_piano, piano_image)
        
        flute_img = np.load(flute_path)
        piano_img = np.load(piano_path)
        
        # add extra dimension for the single channel
        flute_img = flute_img[None, :]
        piano_img = piano_img[None, :]
        
        # standardize the scale
        flute_img = (flute_img - self.offsetFlute) / self.divFlute
        piano_img = (piano_img - self.offsetPiano) / self.divPiano        

        return flute_img, piano_img

In [8]:
def save_checkpoint(model, optimizer, filename="my_checkpoint.pth.tar"):    
    checkpoint = {
        "model": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)

In [9]:
################################
# paths to directories
################################

curDir = os.getcwd()
trainSetDir = f'{curDir}/../../dataSuperSet/processedData/trainSet'
fluteTrainSetDir = f'{trainSetDir}/flute/cqtChunks'
pianoTrainSetDir = f'{trainSetDir}/piano/cqtChunks'

# create directories to store checkpoint outputs
checkPointDir = f'{curDir}/checkPoints'
checkPointModelDir = f'{checkPointDir}/models'
checkPointImageDir = f'{checkPointDir}/images'
checkPointLossTrackingDir = f'{checkPointDir}/lossTracking'

os.system(f'mkdir -p {checkPointDir}')
os.system(f'mkdir -p {checkPointModelDir}')
os.system(f'mkdir -p {checkPointImageDir}')
os.system(f'mkdir -p {checkPointLossTrackingDir}')

0

In [10]:
################################
# datasets
################################

# create dataset
dataset = PianoFluteDataset(pianoTrainSetDir, fluteTrainSetDir, STANDARDIZATION_STYLE)

# create dataloader
loader = torch.utils.data.DataLoader(dataset, batch_size = BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

Using uniform assumption...


In [11]:
#########################################
# instantiate networks and optimizers
#########################################

# discriminator piano
disc_P = Discriminator(in_channels=1).to(DEVICE)

# discriminator flute
disc_F = Discriminator(in_channels=1).to(DEVICE)

# generator piano
gen_P = Generator(img_channels=1, num_residuals = NUM_RESIDUALS).to(DEVICE)

# generator piano
gen_F = Generator(img_channels=1, num_residuals = NUM_RESIDUALS).to(DEVICE)

# optimizer discriminator
opt_disc = optim.Adam(list(disc_P.parameters()) + list(disc_F.parameters()), lr = LEARNING_RATE, betas=(ADAM_BETA1, ADAM_BETA2))

# optimizer generator
opt_gen  = optim.Adam(list(gen_P.parameters())  + list(gen_F.parameters()),  lr = LEARNING_RATE, betas=(ADAM_BETA1, ADAM_BETA2))


# Losses

# For cycle consistency and identity loss
L1 = nn.L1Loss() 

# adversarial loss
mse = nn.MSELoss()


In [12]:
################################
# training
################################

generatorLossProgression = np.zeros(NUM_EPOCHS)
discriminatorLossProgression = np.zeros(NUM_EPOCHS)
identityLossProgression = np.zeros(NUM_EPOCHS)
cycleLossProgression = np.zeros(NUM_EPOCHS)

for epoch in range(NUM_EPOCHS):    
    
    for idx, (flute, piano) in enumerate(loader):
        
        # move data to device
        piano = piano.to(DEVICE)
        flute = flute.to(DEVICE)
                
        
        ##############################
        # Discriminator training
        ##############################
        
        # piano generator output 
        fake_piano = gen_P (flute)        
        
        # piano discriminator
        D_P_real = disc_P(piano)
        D_P_fake = disc_P(fake_piano.detach())                
        
        # real = 1, fake = 0
        D_P_real_loss = mse(D_P_real, torch.ones_like(D_P_real))
        D_P_fake_loss = mse(D_P_fake, torch.zeros_like(D_P_real))
                
        
        # total piano discriminator loss
        Disc_piano_loss = D_P_real_loss + D_P_fake_loss

        # flute generator output
        fake_flute = gen_F(piano)
        
        # flute discriminator
        D_F_real = disc_F(flute)
        D_F_fake = disc_F(fake_flute.detach())
        
        # real = 1, fake = 0
        D_F_real_loss = mse(D_F_real, torch.ones_like(D_F_real))
        D_F_fake_loss = mse(D_F_fake, torch.zeros_like(D_F_real))
        
        # total flute discriminator loss
        Disc_flute_loss = D_F_real_loss + D_F_fake_loss

        # Overall discriminator loss
        D_loss = (Disc_flute_loss + Disc_piano_loss)/2

        # zero out the gradients
        opt_disc.zero_grad()
        
        # backprop
        D_loss.backward()
        
        # update discriminator params
        opt_disc.step()        
        
        ##############################
        # Generator training
        ##############################
        
        # Adversarial Loss for both generators
        D_P_fake = disc_P(fake_piano)
        D_F_fake = disc_F(fake_flute)
        
        # generator wants fake to be detected real
        loss_G_F = mse(D_F_fake, torch.ones_like(D_F_fake))
        loss_G_P = mse(D_P_fake, torch.ones_like(D_P_fake))

        # Cycle Loss
        cycle_piano = gen_P(fake_flute)
        cycle_flute = gen_F(fake_piano)
        cycle_piano_loss = L1(piano, cycle_piano)
        cycle_flute_loss = L1(flute, cycle_flute)
        
        cycle_loss = cycle_flute_loss + cycle_piano_loss

        # Identity Loss
        identity_flute = gen_F(flute)
        identity_piano = gen_P(piano)
        identity_piano_loss = L1(piano, identity_piano)
        identity_flute_loss = L1(flute, identity_flute)
        
        identity_loss = identity_flute_loss + identity_piano_loss

        # Overall Generator Loss
        G_loss = loss_G_F + loss_G_P + cycle_loss*LAMBDA_CYCLE + identity_loss*LAMBDA_IDENTITY

        # zero out the gradients
        opt_gen.zero_grad()
        
        # backprop
        G_loss.backward()
        
        # update generator params
        opt_gen.step()
        
        if idx % 100 == 0:
            
            # print the current losses
            print (f'[epoch = {epoch}, idx = {idx}]:   D_loss = {D_loss.item()}, \t G_loss = {G_loss.item()}, \t identity_loss = {identity_loss.item()}, \t cycle_loss = {cycle_loss.item()}')            
            
        if idx % 1250 == 0:      
            
            # save the fake piano and flute np array of a sample            
            np.save(f'{checkPointImageDir}/originalPiano__idx_{idx}__{fileTag}.npy', np.squeeze(piano.detach().cpu().numpy()[0,:]))
            np.save(f'{checkPointImageDir}/fakeFlute__idx_{idx}__{fileTag}.npy', np.squeeze(fake_flute.detach().cpu().numpy()[0,:]))            
            np.save(f'{checkPointImageDir}/originalFlute___idx_{idx}__{fileTag}.npy', np.squeeze(flute.detach().cpu().numpy()[0,:]))
            np.save(f'{checkPointImageDir}/fakePiano__idx_{idx}__{fileTag}.npy', np.squeeze(fake_piano.detach().cpu().numpy()[0,:]))
        
        
        # aggregate loss over all batches
        discriminatorLossProgression[epoch] += D_loss.item()
        generatorLossProgression[epoch] += G_loss.item()
        identityLossProgression[epoch] += identity_loss.item()
        cycleLossProgression[epoch] += cycle_loss.item()
    
    # average loss over the length of dataset
    discriminatorLossProgression[epoch] = discriminatorLossProgression[epoch] / len(dataset)
    generatorLossProgression[epoch] = generatorLossProgression[epoch] / len(dataset)
    identityLossProgression[epoch] = identityLossProgression[epoch] / len(dataset)
    cycleLossProgression[epoch] = cycleLossProgression[epoch] / len(dataset)
    
    # save the model and current losses
    if SAVE_CHECKPOINTS:
        save_checkpoint(gen_P, opt_gen, filename = f'{checkPointModelDir}/genp__{fileTag}.pth.tar')
        save_checkpoint(gen_F, opt_gen, filename = f'{checkPointModelDir}/genf__{fileTag}.pth.tar')
        save_checkpoint(disc_P, opt_disc, filename = f'{checkPointModelDir}/criticp__{fileTag}.pth.tar')
        save_checkpoint(disc_F, opt_disc, filename = f'{checkPointModelDir}/criticf__{fileTag}.pth.tar')
    
        # save the lossProgressions
        np.save(f'{checkPointLossTrackingDir}/discriminatorLossProgression__{fileTag}.npy', discriminatorLossProgression)
        np.save(f'{checkPointLossTrackingDir}/generatorLossProgression__{fileTag}.npy', generatorLossProgression)
        np.save(f'{checkPointLossTrackingDir}/identityLossProgression__{fileTag}.npy', identityLossProgression)
        np.save(f'{checkPointLossTrackingDir}/cycleLossProgression__{fileTag}.npy', cycleLossProgression)
    
    




[epoch = 0, idx = 0]:   D_loss = 0.5190601348876953, 	 G_loss = 17.873247146606445, 	 identity_loss = 1.5708121061325073, 	 cycle_loss = 1.5711570978164673
[epoch = 0, idx = 100]:   D_loss = 0.3823412358760834, 	 G_loss = 2.0176351070404053, 	 identity_loss = 0.10050852596759796, 	 cycle_loss = 0.09537442773580551
[epoch = 0, idx = 200]:   D_loss = 0.32416439056396484, 	 G_loss = 5.209186553955078, 	 identity_loss = 0.3782009482383728, 	 cycle_loss = 0.35637950897216797
[epoch = 0, idx = 300]:   D_loss = 0.22514678537845612, 	 G_loss = 2.96627140045166, 	 identity_loss = 0.12279185652732849, 	 cycle_loss = 0.15542156994342804
[epoch = 0, idx = 400]:   D_loss = 0.24173375964164734, 	 G_loss = 2.7518320083618164, 	 identity_loss = 0.12341061234474182, 	 cycle_loss = 0.12459419667720795
[epoch = 0, idx = 500]:   D_loss = 0.36445504426956177, 	 G_loss = 6.294418811798096, 	 identity_loss = 0.4570660889148712, 	 cycle_loss = 0.44537419080734253
[epoch = 0, idx = 600]:   D_loss = 0.407689630

[epoch = 0, idx = 5200]:   D_loss = 0.28613969683647156, 	 G_loss = 1.8329795598983765, 	 identity_loss = 0.062386371195316315, 	 cycle_loss = 0.05452865734696388
[epoch = 0, idx = 5300]:   D_loss = 0.2489027976989746, 	 G_loss = 3.8765993118286133, 	 identity_loss = 0.4907921850681305, 	 cycle_loss = 0.19293692708015442
[epoch = 1, idx = 0]:   D_loss = 0.3657815754413605, 	 G_loss = 1.6116875410079956, 	 identity_loss = 0.06652388721704483, 	 cycle_loss = 0.05880359560251236
[epoch = 1, idx = 100]:   D_loss = 0.26611053943634033, 	 G_loss = 3.5899341106414795, 	 identity_loss = 0.34900787472724915, 	 cycle_loss = 0.1884329468011856
[epoch = 1, idx = 200]:   D_loss = 0.30960094928741455, 	 G_loss = 2.0613131523132324, 	 identity_loss = 0.06353692710399628, 	 cycle_loss = 0.06001468747854233
[epoch = 1, idx = 300]:   D_loss = 0.3850080966949463, 	 G_loss = 2.2292635440826416, 	 identity_loss = 0.10402233898639679, 	 cycle_loss = 0.08358635753393173
[epoch = 1, idx = 400]:   D_loss = 0.4

[epoch = 1, idx = 4900]:   D_loss = 0.4071611762046814, 	 G_loss = 1.7822118997573853, 	 identity_loss = 0.08846161514520645, 	 cycle_loss = 0.06702212244272232
[epoch = 1, idx = 5000]:   D_loss = 0.4215311110019684, 	 G_loss = 1.648869514465332, 	 identity_loss = 0.05688926950097084, 	 cycle_loss = 0.05076834559440613
[epoch = 1, idx = 5100]:   D_loss = 0.40565308928489685, 	 G_loss = 1.581490159034729, 	 identity_loss = 0.05967816710472107, 	 cycle_loss = 0.05269952118396759
[epoch = 1, idx = 5200]:   D_loss = 0.46102407574653625, 	 G_loss = 1.6568368673324585, 	 identity_loss = 0.06487749516963959, 	 cycle_loss = 0.06496571749448776
[epoch = 1, idx = 5300]:   D_loss = 0.4082600474357605, 	 G_loss = 1.4470884799957275, 	 identity_loss = 0.056653942912817, 	 cycle_loss = 0.04656952619552612
[epoch = 2, idx = 0]:   D_loss = 0.4492861032485962, 	 G_loss = 1.6166170835494995, 	 identity_loss = 0.07625381648540497, 	 cycle_loss = 0.07189688086509705
[epoch = 2, idx = 100]:   D_loss = 0.50

[epoch = 2, idx = 4600]:   D_loss = 0.4774971604347229, 	 G_loss = 1.2469375133514404, 	 identity_loss = 0.042050063610076904, 	 cycle_loss = 0.04777035117149353
[epoch = 2, idx = 4700]:   D_loss = 0.5033745765686035, 	 G_loss = 1.2263948917388916, 	 identity_loss = 0.03973674774169922, 	 cycle_loss = 0.04700557887554169
[epoch = 2, idx = 4800]:   D_loss = 0.40928414463996887, 	 G_loss = 1.4920154809951782, 	 identity_loss = 0.059008367359638214, 	 cycle_loss = 0.05254107713699341
[epoch = 2, idx = 4900]:   D_loss = 0.4481261968612671, 	 G_loss = 1.3094892501831055, 	 identity_loss = 0.054706424474716187, 	 cycle_loss = 0.046284519135951996
[epoch = 2, idx = 5000]:   D_loss = 0.4543225169181824, 	 G_loss = 1.2619901895523071, 	 identity_loss = 0.05677735060453415, 	 cycle_loss = 0.03986230492591858
[epoch = 2, idx = 5100]:   D_loss = 0.4871302545070648, 	 G_loss = 1.130094289779663, 	 identity_loss = 0.03586959466338158, 	 cycle_loss = 0.04161129146814346
[epoch = 2, idx = 5200]:   D_l

[epoch = 3, idx = 4300]:   D_loss = 0.4307709038257599, 	 G_loss = 1.3399062156677246, 	 identity_loss = 0.03796110302209854, 	 cycle_loss = 0.04601643234491348
[epoch = 3, idx = 4400]:   D_loss = 0.41827404499053955, 	 G_loss = 1.3260738849639893, 	 identity_loss = 0.041453734040260315, 	 cycle_loss = 0.04163014516234398
[epoch = 3, idx = 4500]:   D_loss = 0.4397515654563904, 	 G_loss = 1.2989269495010376, 	 identity_loss = 0.041916314512491226, 	 cycle_loss = 0.044430140405893326
[epoch = 3, idx = 4600]:   D_loss = 0.4882332682609558, 	 G_loss = 1.220016360282898, 	 identity_loss = 0.03867530822753906, 	 cycle_loss = 0.04696977511048317
[epoch = 3, idx = 4700]:   D_loss = 0.42156338691711426, 	 G_loss = 1.24589204788208, 	 identity_loss = 0.03749127686023712, 	 cycle_loss = 0.03655357286334038
[epoch = 3, idx = 4800]:   D_loss = 0.44461461901664734, 	 G_loss = 1.2559252977371216, 	 identity_loss = 0.04253820329904556, 	 cycle_loss = 0.04322770982980728
[epoch = 3, idx = 4900]:   D_lo

...

[epoch = 5, idx = 3700]:   D_loss = 0.46183186769485474, 	 G_loss = 1.1819103956222534, 	 identity_loss = 0.027702350169420242, 	 cycle_loss = 0.041296400129795074
[epoch = 5, idx = 3800]:   D_loss = 0.3887971043586731, 	 G_loss = 1.3874850273132324, 	 identity_loss = 0.047323182225227356, 	 cycle_loss = 0.04464484751224518
[epoch = 5, idx = 3900]:   D_loss = 0.49360013008117676, 	 G_loss = 1.2536365985870361, 	 identity_loss = 0.03076813742518425, 	 cycle_loss = 0.042707301676273346
[epoch = 5, idx = 4000]:   D_loss = 0.464726984500885, 	 G_loss = 1.0847827196121216, 	 identity_loss = 0.025611326098442078, 	 cycle_loss = 0.03870079666376114
[epoch = 5, idx = 4100]:   D_loss = 0.5280382633209229, 	 G_loss = 1.1136342287063599, 	 identity_loss = 0.04391709715127945, 	 cycle_loss = 0.03483537584543228
[epoch = 5, idx = 4200]:   D_loss = 0.4427831768989563, 	 G_loss = 1.3287376165390015, 	 identity_loss = 0.03962552547454834, 	 cycle_loss = 0.047829870134592056
[epoch = 5, idx = 4300]:   

[epoch = 6, idx = 3400]:   D_loss = 0.41355907917022705, 	 G_loss = 1.4125913381576538, 	 identity_loss = 0.033021777868270874, 	 cycle_loss = 0.04917088523507118
[epoch = 6, idx = 3500]:   D_loss = 0.5164274573326111, 	 G_loss = 1.0652209520339966, 	 identity_loss = 0.030641209334135056, 	 cycle_loss = 0.03278094157576561
[epoch = 6, idx = 3600]:   D_loss = 0.44001662731170654, 	 G_loss = 1.0796672105789185, 	 identity_loss = 0.02795882150530815, 	 cycle_loss = 0.03336700052022934
[epoch = 6, idx = 3700]:   D_loss = 0.4117968678474426, 	 G_loss = 1.2212179899215698, 	 identity_loss = 0.03024948388338089, 	 cycle_loss = 0.041376471519470215
[epoch = 6, idx = 3800]:   D_loss = 0.47178584337234497, 	 G_loss = 1.3454800844192505, 	 identity_loss = 0.03636304289102554, 	 cycle_loss = 0.05350179225206375
[epoch = 6, idx = 3900]:   D_loss = 0.46069154143333435, 	 G_loss = 1.180752158164978, 	 identity_loss = 0.02546301856637001, 	 cycle_loss = 0.03648248314857483
[epoch = 6, idx = 4000]:   D

[epoch = 7, idx = 3100]:   D_loss = 0.45026838779449463, 	 G_loss = 1.1767364740371704, 	 identity_loss = 0.02531370148062706, 	 cycle_loss = 0.038916051387786865
[epoch = 7, idx = 3200]:   D_loss = 0.4496954083442688, 	 G_loss = 1.2001134157180786, 	 identity_loss = 0.02562558278441429, 	 cycle_loss = 0.03871915861964226
[epoch = 7, idx = 3300]:   D_loss = 0.4697013795375824, 	 G_loss = 1.100441813468933, 	 identity_loss = 0.02359088696539402, 	 cycle_loss = 0.035427019000053406
[epoch = 7, idx = 3400]:   D_loss = 0.44129014015197754, 	 G_loss = 1.3100591897964478, 	 identity_loss = 0.03120383992791176, 	 cycle_loss = 0.04128967970609665
[epoch = 7, idx = 3500]:   D_loss = 0.47091740369796753, 	 G_loss = 1.0515104532241821, 	 identity_loss = 0.02335149422287941, 	 cycle_loss = 0.03353797644376755
[epoch = 7, idx = 3600]:   D_loss = 0.4486549496650696, 	 G_loss = 1.3350011110305786, 	 identity_loss = 0.039865028113126755, 	 cycle_loss = 0.03946685418486595
[epoch = 7, idx = 3700]:   D_

[epoch = 8, idx = 2800]:   D_loss = 0.46614494919776917, 	 G_loss = 1.1481571197509766, 	 identity_loss = 0.03659728914499283, 	 cycle_loss = 0.03865399211645126
[epoch = 8, idx = 2900]:   D_loss = 0.47796815633773804, 	 G_loss = 1.0532861948013306, 	 identity_loss = 0.024391531944274902, 	 cycle_loss = 0.03598041087388992
[epoch = 8, idx = 3000]:   D_loss = 0.4924302101135254, 	 G_loss = 1.0945711135864258, 	 identity_loss = 0.024026665836572647, 	 cycle_loss = 0.03514140471816063
[epoch = 8, idx = 3100]:   D_loss = 0.42342764139175415, 	 G_loss = 1.2650649547576904, 	 identity_loss = 0.02870882675051689, 	 cycle_loss = 0.045086052268743515
[epoch = 8, idx = 3200]:   D_loss = 0.4177573323249817, 	 G_loss = 1.1143896579742432, 	 identity_loss = 0.023091381415724754, 	 cycle_loss = 0.031742073595523834
[epoch = 8, idx = 3300]:   D_loss = 0.4370262026786804, 	 G_loss = 1.1511234045028687, 	 identity_loss = 0.021291669458150864, 	 cycle_loss = 0.04046889394521713
[epoch = 8, idx = 3400]: 

[epoch = 9, idx = 2500]:   D_loss = 0.3858007788658142, 	 G_loss = 1.3732131719589233, 	 identity_loss = 0.029068175703287125, 	 cycle_loss = 0.04138953238725662
[epoch = 9, idx = 2600]:   D_loss = 0.4659661650657654, 	 G_loss = 1.0461958646774292, 	 identity_loss = 0.023805689066648483, 	 cycle_loss = 0.030843134969472885
[epoch = 9, idx = 2700]:   D_loss = 0.431890606880188, 	 G_loss = 1.1681475639343262, 	 identity_loss = 0.02167404070496559, 	 cycle_loss = 0.03464668616652489
[epoch = 9, idx = 2800]:   D_loss = 0.4677564203739166, 	 G_loss = 1.0667446851730347, 	 identity_loss = 0.026079075410962105, 	 cycle_loss = 0.03377863019704819
[epoch = 9, idx = 2900]:   D_loss = 0.43049830198287964, 	 G_loss = 1.216578722000122, 	 identity_loss = 0.027031470090150833, 	 cycle_loss = 0.03628263622522354
[epoch = 9, idx = 3000]:   D_loss = 0.4791945219039917, 	 G_loss = 1.10045325756073, 	 identity_loss = 0.024768196046352386, 	 cycle_loss = 0.03066195175051689
[epoch = 9, idx = 3100]:   D_lo